The following example notebook implements standard diffusion
with a simple CNN model to generate realistic MNIST digits.

This is a modified implementation of `minDiffusion`
which implements [DDPM](https://arxiv.org/abs/2006.11239).

To run this example notebook,
install requirements as in `requirements.txt` (for example, `pip install -r requirements.txt`).
You may also wish to follow system-dependent PyTorch instructions
[here](https://pytorch.org/) to install accelerated
versions of PyTorch, but note they are not needed
(I am testing this on my laptop).

If you do use accelerated hardware, make sure that your code
is still compatible with CPU-only installs.

First, let's create a folder to store example images:

In [3]:
!mkdir -p contents

In [4]:
from typing import Dict, Tuple

import numpy as np
import torch
import torch.nn as nn
from accelerate import Accelerator
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image, make_grid

/Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <85A36C65-3F71-3C3B-B529-961AE17DBE73> /Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <44DEDA27-4DE9-3D4A-8EDE-5AA72081319F> /Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherw

The following function creates a DDPM training schedule for use when evaluating
and training the diffusion model:

In [5]:

# I can try to change linear noise schedule to geometric noise schedule


def ddpm_schedules(beta1: float, beta2: float, T: int) -> Dict[str, torch.Tensor]:
    """Returns pre-computed schedules for DDPM sampling with a linear noise schedule."""
    
    # beta1 and beta2 are the start and end values of the noise schedule
    # T is the number of steps in the schedule
    assert beta1 < beta2 < 1.0, "beta1 and beta2 must be in (0, 1)"

    beta_t = (beta2 - beta1) * torch.arange(0, T + 1, dtype=torch.float32) / T + beta1  # Linear schedule
    alpha_t = torch.exp(torch.cumsum(torch.log(1 - beta_t), dim=0))  # Cumprod in log-space (better precision)

    return {"beta_t": beta_t, "alpha_t": alpha_t}


# Defining a geometric noise schedule

def ddp_geo_schedule(beta1: float, beta2: float, T:int) -> Dict[str, torch.Tensor]:
    """_summary_

    Args:
        beta1 (float): The start value of the noise schedule
        beat2 (float): The end value of the noise schedule
        T (int): Number of steps in the schedule

    Returns:
        Dict[str, torch.Tensor]: Pre-computed schedules for DDPM sampling with a geometric noise schedule
    """
    
    assert beta1 < beta2 < 1.0, "beta1 and beta2 must be in (0, 1)"
    
    beta_t = beta1 * (beta2 / beta1) ** (torch.arange(0, T + 1, dtype=torch.float32) / T)  # Geometric schedule
    alpha_t = torch.exp(torch.cumsum(torch.log(1 - beta_t), dim=0))  # Cumprod in log-space (better precision)
    
    return {"beta_t": beta_t, "alpha_t": alpha_t}

# Defining my own noise schedule (Small rate first --> Fast rate --> Small rate) Just like the learning rate schedule
def ddp_custom_schedule(beta1: float, beta2: float, T:int) -> Dict[str, torch.Tensor]:
    

Next, we create a simple 2D convolutional neural network. This network
is essentially going to try to estimate the diffusion process --- we
can then use this network to generate realistic images.

First, we create a single CNN block which we will stack to create the
full network. We use `LayerNorm` for stable training and no batch dependence.

In [6]:
class CNNBlock(nn.Module):
    def __init__(   #constructor
        self,
        in_channels,  # Number of input channels -- depth (e.g. 3 for an RGB image, 1 for a grayscale image)
        out_channels,  # Number of output channels -- also the number of kernels in the convolutional layer
        *,
        expected_shape,  # Dimensions of the input tensor (e.g. (3, 64, 64) for an RGB image with size 64x64)
        act=nn.GELU,
        kernel_size=7,  # Size of the kernel, square kernel is assumed
    ):
        super().__init__()  # Initialize the parent class

        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=kernel_size // 2),
            nn.LayerNorm((out_channels, *expected_shape)),
            act()  # Activation function -- GELU
        )

    def forward(self, x):  
        return self.net(x)

We then create the full CNN model, which is a stack of these blocks
according to the `n_hidden` tuple, which specifies the number of
channels at each hidden layer.

In [7]:
class CNN(nn.Module):
    def __init__(
        self,
        in_channels,
        expected_shape=(28, 28),  # Dimensions of the input tensor for MNIST images
        n_hidden=(64, 128, 64),   # Number of output channels at each hidden layer
        kernel_size=7,
        last_kernel_size=3,
        time_embeddings=16,
        act=nn.GELU,
    ) -> None:
        super().__init__()
        last = in_channels

        self.blocks = nn.ModuleList() # List of CNN blocks
        for hidden in n_hidden:
            self.blocks.append(
                CNNBlock(
                    last,  # Number of input channels
                    hidden,
                    expected_shape=expected_shape,
                    kernel_size=kernel_size,
                    act=act,
                )
            )
            last = hidden  # Update the number of input channels for the next layer --> output for previous layer is input for next layer

        # The final layer, we use a regular Conv2d to get the
        # correct scale and shape (and avoid applying the activation)
        self.blocks.append(  # Append the final layer to the list of CNN blocks
            nn.Conv2d(  # Output layer should match the input layer
                last,
                in_channels,
                last_kernel_size,
                padding=last_kernel_size // 2,
            )
        )

        ## This part is literally just to put the single scalar "t" into the CNN
        ## in a nice, high-dimensional way:
        self.time_embed = nn.Sequential(
            nn.Linear(time_embeddings * 2, 128), act(),
            nn.Linear(128, 128), act(),
            nn.Linear(128, 128), act(),
            nn.Linear(128, n_hidden[0]),
        )
        frequencies = torch.tensor(
            [0] + [2 * np.pi * 1.5**i for i in range(time_embeddings - 1)]
        )
        self.register_buffer("frequencies", frequencies)

    def time_encoding(self, t: torch.Tensor) -> torch.Tensor:
        phases = torch.concat(
            (
                torch.sin(t[:, None] * self.frequencies[None, :]),
                torch.cos(t[:, None] * self.frequencies[None, :]) - 1,
            ),
            dim=1,
        )

        return self.time_embed(phases)[:, :, None, None]

    def forward(self, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        # Shapes of input:
        #    x: (batch, chan, height, width)
        #    t: (batch,)

        embed = self.blocks[0](x)  # Apply the first CNN block
        # ^ (batch, n_hidden[0], height, width)

        # Add information about time along the diffusion process
        #  (Providing this information by superimposing in latent space)
        embed += self.time_encoding(t)
        #         ^ (batch, n_hidden[0], 1, 1) - thus, broadcasting
        #           to the entire spatial domain

        for block in self.blocks[1:]:  # Apply the remaining CNN blocks (2 and 3 in this case)
            embed = block(embed)

        return embed

Next, we define the actual diffusion model, which specifies the training
schedule, takes an arbitrary model for estimating the
diffusion process (such as the CNN above),
and computes the corresponding loss (as well as generating samples).

In [8]:
class DDPM(nn.Module):
    def __init__(
        self,
        gt,  # The model to predict the noide at each step
        betas: Tuple[float, float],  # Start and end values of the noise schedule
        n_T: int,   # Number of steps in diffusion process.
        criterion: nn.Module = nn.MSELoss(),  # Loss function to use during training
    ) -> None:
        super().__init__()

        self.gt = gt

        noise_schedule = ddpm_schedules(betas[0], betas[1], n_T)  # Pre-computed schedule dictionary for DDPM sampling

        # `register_buffer` will track these tensors for device placement, but
        # not store them as model parameters. This is useful for constants.
        self.register_buffer("beta_t", noise_schedule["beta_t"])  # Buffers are like parameters, but not updated by optimizer
        self.beta_t  # Exists! Set by register_buffer
        self.register_buffer("alpha_t", noise_schedule["alpha_t"])
        self.alpha_t

        self.n_T = n_T
        self.criterion = criterion

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Algorithm 18.1 in Prince"""

        t = torch.randint(1, self.n_T, (x.shape[0],), device=x.device)
        eps = torch.randn_like(x)  # eps ~ N(0, 1)
        alpha_t = self.alpha_t[t, None, None, None]  # Get right shape for broadcasting

        z_t = torch.sqrt(alpha_t) * x + torch.sqrt(1 - alpha_t) * eps
        # This is the z_t, which is sqrt(alphabar) x_0 + sqrt(1-alphabar) * eps
        # We should predict the "error term" from this z_t. Loss is what we return.

        return self.criterion(eps, self.gt(z_t, t / self.n_T))

    def sample(self, n_sample: int, size, device) -> torch.Tensor:
        """Algorithm 18.2 in Prince"""

        _one = torch.ones(n_sample, device=device)
        z_t = torch.randn(n_sample, *size, device=device)
        for i in range(self.n_T, 0, -1):
            alpha_t = self.alpha_t[i]
            beta_t = self.beta_t[i]

            # First line of loop:
            z_t -= (beta_t / torch.sqrt(1 - alpha_t)) * self.gt(z_t, (i/self.n_T) * _one)
            z_t /= torch.sqrt(1 - beta_t)

            if i > 1:
                # Last line of loop:
                z_t += torch.sqrt(beta_t) * torch.randn_like(z_t)
            # (We don't add noise at the final step - i.e., the last line of the algorithm)

        return z_t

We will run this on MNIST. We perform some basic preprocessing, and set up the data loader:

In [9]:
tf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0))])
dataset = MNIST("./data", train=True, download=True, transform=tf)
training_dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=4, drop_last=True)

In [ ]:
# Creating a validation dataset

validation_dataset = MNIST("./data", train=False, download=True, transform=tf)
validation_dataloader = DataLoader(validation_dataset, batch_size=128, shuffle = True, num_workers=4, drop_last=True)

We create our model with a given choice of hidden layers and activation function. We also choose a learning rate.

In [10]:
gt = CNN(in_channels=1, expected_shape=(28, 28), n_hidden=(16, 32, 32, 16), act=nn.GELU)  #gray scale image
# For testing: (16, 32, 32, 16)
# For more capacity (for example): (64, 128, 256, 128, 64)
ddpm = DDPM(gt=gt, betas=(1e-4, 0.02), n_T=1000)
optim = torch.optim.Adam(ddpm.parameters(), lr=2e-4)

We could set up a GPU if we have one, which is done below.

Here, we use HuggingFace's `accelerate` library, which abstracts away all the `.to(device)` calls for us.
This lets us focus on the model itself rather than data movement.
It also does a few other tricks to speed up calculations.

PyTorch Lightning, which we discussed during the course, is another option that also handles a lot more, but is a bit heavyweight.
`accelerate` is a simpler option closer to raw PyTorch.
However, if you prefer, you could choose to use Lightning for the coursework!

In [11]:
accelerator = Accelerator()

# We wrap our model, optimizer, and dataloaders with `accelerator.prepare`,
# which lets HuggingFace's Accelerate handle the device placement and gradient accumulation.
ddpm, optim, training_dataloader = accelerator.prepare(ddpm, optim, dataloader)

First, let's just make sure this works:

In [12]:
for x, _ in training_dataloader:
    break

with torch.no_grad():
    ddpm(x)

/Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <85A36C65-3F71-3C3B-B529-961AE17DBE73> /Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <44DEDA27-4DE9-3D4A-8EDE-5AA72081319F> /Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to

In [ ]:
# Looking at the size of the training_loader and validation_loader
print("The number of batches in the training_loader is: ", len(training_dataloader))
print("The number of batches in the validation_loader is: ", len(validation_dataloader))

Now, let's train it. You can exit early by interrupting the kernel. Images
are saved to the `contents` folder.

In [13]:
n_epoch = 100
training_losses = []
validation_losses = []


for i in range(n_epoch):  # Loop over the epochs
    ddpm.train()  # Set the model to training mode

    pbar = tqdm(training_dataloader)  # Wrap our loop with a visual progress bar
    for x, _ in pbar:  # Loop over the #batches = len(training_dataloader) --> 128 images per batch
        optim.zero_grad()  # Clear gradients before each step

        loss = ddpm(x)  # Forward pass to calculate the loss

        loss.backward()
        # ^Technically should be `accelerator.backward(loss)` but not necessary for local training

        training_losses.append(loss.item())
        avg_loss = np.average(training_losses[min(len(training_losses)-100, 0):])
        pbar.set_description(f"loss: {avg_loss:.3g}")  # Show running average of loss in progress bar

        optim.step()  # Update the model's parameters based on the calculated gradients

    ddpm.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No need to calculate gradients when evaluating the model
        
        #_______Sample after each epoch_______   
        
        xh = ddpm.sample(16, (1, 28, 28), accelerator.device)  # Can get device explicitly with `accelerator.device`
        grid = make_grid(xh, nrow=4)

        # Save samples to `./contents` directory
        save_image(grid, f"./contents/ddpm_sample_{i:04d}.png")

        # save model
        torch.save(ddpm.state_dict(), f"./ddpm_mnist.pth")
        
        
        #_______Validation phase_______
        val_loss = 0.0  # Initialize the validation loss
        for x_val, _ in validation_dataloader:
            
            # Forward pass on validation batch
            val_loss += ddpm(x_val)
            
        # Calculate the average validation loss over each batch
        #val_loss /= len(validation_dataloader)
    
            validation_losses.append(val_loss.item())
            
      
           
            
            
            


  0%|          | 0/468 [00:00<?, ?it/s]/Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <85A36C65-3F71-3C3B-B529-961AE17DBE73> /Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <44DEDA27-4DE9-3D4A-8EDE-5AA72081319F> /Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/mertdaloglu/anaconda3/envs/project/lib/python3.11/site-packages/torchvision

KeyboardInterrupt: 

In [14]:
import matplotlib.pyplot as plt

# Plotting the loss curve and validation curve as a function of the number of iterations

plt.figure(figsize(10,5))
plt.plot(training_losses, label="Training Loss")
plt.plot(validation_losses, label="Validation Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()
plt.show()


In [16]:
# Making a GIF of the samples

import imageio
import glob

images = []
for filename in sorted(glob.glob("./contents/ddpm_sample_*.png")):
    images.append(imageio.imread(filename))
    
imageio.mimsave("./gifs/ddpm_samples.gif", images, fps=2)

/var/folders/gk/dsypcq4n5hg6hh5qdss0ly540000gn/T/ipykernel_1280/4065097656.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
